In [9]:
#Importing modules
import pyodbc
import json
import collections

In [10]:
class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, decimal.Decimal):
            return float(o)
        return super(DecimalEncoder, self).default(o)

In [11]:
#MSSQL connection
connstr = 'DRIVER={FreeTDS};SERVER=msdb2.ivy-support.com;PORT=1433;DATABASE=sd_castrol;UID=sd_castrol_sauser;PWD=sd_castrol_sauseradfasd;TDS_Version=7.2;'
conn = pyodbc.connect(connstr)
cursor = conn.cursor()

In [12]:
cursor.execute("""
            SELECT 
ID.InvoiceNo "InvNo" 
,ID.ProductID "ProductId"
,PM.AProductName "ProductName"
,SUM(ID.Qty) "InvoicedQty"
,ID.Rate "UnitPrice"
,SUM(ID.Discount) "Discount"
,SUM(ID.NetRate) "NetAmt"
FROM InvoiceDetails ID
LEFT JOIN ProductMaster PM ON ID.ProductID=PM.Old_ProductCode
GROUP BY ID.InvoiceNo,ID.ProductID,PM.AProductName,ID.Rate 
            """)
 
rows = cursor.fetchall()

In [13]:
# Convert query to row arrays
 
rowarray_list = []
for row in rows:
    t = (row.InvNo, row.ProductId, row.ProductName, row.InvoicedQty,float(row.Discount),
         float(row.UnitPrice),float(row.NetAmt))
    rowarray_list.append(t)

with open('invoice_ret_rowarrays.json', 'w') as outfile:  
    json.dump(rowarray_list, outfile)
    
#print f, j

In [16]:
# Convert query to objects of key-value pairs
 
objects_list = []
for row in rows:
    d = collections.OrderedDict()
    d['InvNo'] = row.InvNo
    d['ProductId'] = row.ProductId
    d['ProductName'] = row.ProductName
    d['InvoicedQty'] = row.InvoicedQty
    d['Discount'] = float(row.Discount)
    d['UnitPrice'] = float(row.UnitPrice)
    d['NetAmt'] = float(row.NetAmt)
    objects_list.append(d)

with open('student_objects.json','w') as outputfile:  
    json.dump(objects_list, outputfile)
#rint f, j

In [17]:
#Closing the connection
conn.close()